In [1]:
import warnings
warnings.filterwarnings('ignore')
import wradlib as wrl
import matplotlib.pyplot as pl
import matplotlib as mpl
from matplotlib.collections import PatchCollection
from matplotlib.colors import from_levels_and_colors
from matplotlib.path import Path
import matplotlib.patches as patches
import matplotlib.cm as cm
#try:
#    get_ipython().magic("matplotlib inline")
#except:
#    pl.ion()
import numpy as np
import datetime as dt
from osgeo import osr
warnings.filterwarnings('ignore')
from satlib import corcor
import os
import time



In [2]:
def subplot_cfad(x1, x2, ZP):
    """
    Plot 4 CFAD
    """
    import matplotlib.pyplot as plt
    m1 = ~np.isnan(x1) & ~np.isnan(x2)
    
    x3 = np.array([]); x4 = np.array([]); x5 = np.array([np.nan])

    hh = np.array([])
    for jj in range(0,35000,100):
        hh = np.append(hh,jj)
        x3 = np.append(x3,np.nanmean(x1[np.where((x2>jj)&(x2<jj+100))]))
        x4 = np.append(x4,np.nanmedian(x1[np.where((x2>jj)&(x2<jj+100))]))
        x0 = x1.copy()
        if x0[np.where((x2>jj)&(x2<jj+100))].size==0:
            x5 = np.append(x5,0)
        else:
            x5 = np.append(x5,np.nanmax(x0[np.where((x2>jj)&(x2<jj+100))]))
    
    print hh.shape, x5.shape, x4.shape
    plt.hist2d(x1[m1], x2[m1],bins=30, cmap='inferno', vmax=40)
    #plt.hexbin(x1[m1], x2[m1],bins=10, cmap='inferno')
    plt.colorbar()
    plt.xlabel('ZH in DBz')
    plt.ylabel('Height in m')
    plt.title('CFAD: '+ ZP)
    plt.plot(x3,hh, linewidth=2, label='mean')
    plt.plot(x4,hh, linewidth=2, label='median')
    #plt.plot(x5[1::],hh, linewidth=2, label='max', color='white')
    # BB detection with cfad?
    #plt.axhline(0, color='white')

    plt.legend(loc='upper right', fontsize=10)
    plt.grid(color='white')
    #plt.close()
    plt.ylim(250,7200)
    #plt.show()



In [3]:
from netCDF4 import Dataset
#from wradlib import get_clip_mask

def read_gpm(filename, bbox):
    scan='NS' # NS
    pr_data = Dataset(filename, mode="r")
    lon = pr_data[scan].variables['Longitude']
    lat = pr_data[scan].variables['Latitude']

    poly = [[bbox['left'], bbox['bottom']],
            [bbox['left'], bbox['top']],
            [bbox['right'], bbox['top']],
            [bbox['right'], bbox['bottom']],
            [bbox['left'], bbox['bottom']]]
    mask = wrl.zonalstats.get_clip_mask(np.dstack((lon[:], lat[:])), poly)
    mask = np.nonzero(np.count_nonzero(mask, axis=1))
    lon = lon[mask]
    lat = lat[mask]

    year = pr_data[scan]['ScanTime'].variables['Year'][mask]
    month = pr_data[scan]['ScanTime'].variables['Month'][mask]
    dayofmonth = pr_data[scan]['ScanTime'].variables['DayOfMonth'][mask]
    # dayofyear = pr_data[scan]['ScanTime'].variables['DayOfYear'][mask]
    hour = pr_data[scan]['ScanTime'].variables['Hour'][mask]
    minute = pr_data[scan]['ScanTime'].variables['Minute'][mask]
    second = pr_data[scan]['ScanTime'].variables['Second'][mask]
    # secondofday = pr_data[scan]['ScanTime'].variables['SecondOfDay'][mask]
    millisecond = pr_data[scan]['ScanTime'].variables['MilliSecond'][mask]
    date_array = zip(year, month, dayofmonth,
                     hour, minute, second,
                     millisecond.astype(np.int32) * 1000)
    pr_time = np.array(
        [dt.datetime(d[0], d[1], d[2], d[3], d[4], d[5], d[6]) for d in
         date_array])

    sfc = pr_data[scan]['PRE'].variables['landSurfaceType'][mask]
    pflag = pr_data[scan]['PRE'].variables['flagPrecip'][mask]

    # bbflag = pr_data[scan]['CSF'].variables['flagBB'][mask]
    zbb = pr_data[scan]['CSF'].variables['heightBB'][mask]
    # print(zbb.dtype)
    bbwidth = pr_data[scan]['CSF'].variables['widthBB'][mask]
    qbb = pr_data[scan]['CSF'].variables['qualityBB'][mask]
    qtype = pr_data[scan]['CSF'].variables['qualityTypePrecip'][mask]
    ptype = pr_data[scan]['CSF'].variables['typePrecip'][mask]

    quality = pr_data[scan]['scanStatus'].variables['dataQuality'][mask]
    ## REFL
    refl = pr_data[scan]['SLV'].variables['zFactorCorrected'][mask]
    #refl = pr_data[scan]['PRE'].variables['zFactorMeasured'][mask]
    # print(pr_data[scan]['SLV'].variables['zFactorCorrected']) 

    zenith = pr_data[scan]['PRE'].variables['localZenithAngle'][mask]

    pr_data.close()

    # Check for bad data
    if max(quality) != 0:
        raise ValueError('GPM contains Bad Data')

    pflag = pflag.astype(np.int8)

    # Determine the dimensions
    ndim = refl.ndim
    if ndim != 3:
        raise ValueError('GPM Dimensions do not match! '
                         'Needed 3, given {0}'.format(ndim))

    tmp = refl.shape
    nscan = tmp[0]
    nray = tmp[1]
    nbin = tmp[2]

    # Reverse direction along the beam
    refl = np.flip(refl, axis=-1)

    # Change pflag=1 to pflag=2 to be consistent with 'Rain certain' in TRMM
    pflag[pflag == 1] = 2

    # Simplify the precipitation types
    ptype = (ptype / 1e7).astype(np.int16)

    # Simplify the surface types
    imiss = (sfc == -9999)
    sfc = (sfc / 1e2).astype(np.int16) + 1
    sfc[imiss] = 0

    # Set a quality indicator for the BB and precip type data
    # TODO: Why is the `quality` variable overwritten?

    quality = np.zeros((nscan, nray), dtype=np.uint8)

    i1 = ((qbb == 0) | (qbb == 1)) & (qtype == 1)
    quality[i1] = 1

    i2 = ((qbb > 1) | (qtype > 2))
    quality[i2] = 2

    gpm_data = {}
    gpm_data.update({'nscan': nscan, 'nray': nray, 'nbin': nbin,
                     'date': pr_time, 'lon': lon, 'lat': lat,
                     'pflag': pflag, 'ptype': ptype, 'zbb': zbb,
                     'bbwidth': bbwidth, 'sfc': sfc, 'quality': quality,
                     'refl': refl, 'zenith': zenith})

    return gpm_data

In [4]:
# define GPM data set
# ----------------------
gpm_file_path = '/automount/ags/velibor/gpmdata/dprV7/2A.GPM.DPR.V7-20170308.20141007-S015721-E032951.003445.V05A.HDF5'
#gpm_file_path = '/automount/ags/velibor/gpmdata/dprV7/2A.GPM.DPR.V7-20170308.20141008-S084747-E102017.003465.V05A.HDF5'
#gpm_file_path = '/automount/ags/velibor/gpmdata/dprV7/2A.GPM.DPR.V7-20170308.20160601-S171836-E185110.012837.V05A.HDF5'
#gpm_file_path = '/automount/ags/velibor/gpmdata/dprV7/2A.GPM.DPR.V7-20170308.20150330-S224903-E002135.006166.V05A.HDF5'
#gpm_file_path = '/automount/ags/velibor/gpmdata/dprV7/2A.GPM.DPR.V7-20170308.20151216-S014742-E032015.010213.V05A.HDF5'

# define matching ground radar file
# ----------------------
gr_file_path = '/automount/radar-archiv/scans/2014/2014-10/2014-10-07/n_ppi_010deg/2014-10-07--02:37:44,00.mvol'
#gr_file_path = '/automount/radar-archiv/scans/2014/2014-10/2014-10-08/n_ppi_010deg/2014-10-08--09:45:00,00.mvol'
#gr_file_path = '/automount/radar/scans/2016/2016-06/2016-06-01/n_ppi_010deg/2016-06-01--17:59:50,00.mvol'
#gr_file_path = '/automount/radar/scans/2015/2015-03/2015-03-30/n_ppi_010deg/2015-03-30--23:30:03,00.mvol'
#gr_file_path = '/automount/radar/scans/2015/2015-12/2015-12-16/n_ppi_010deg/2015-12-16--02:45:01,00.mvol'

gpm_file = wrl.util.get_wradlib_data_file(gpm_file_path)
gr2gpm_file = wrl.util.get_wradlib_data_file(gr_file_path)

In [5]:
# Space-born precipitation radar parameters
sr_pars = {"gpm": {
    "zt": 407000.,  # orbital height of GPM                 APPROXIMATION!
    "dr": 125.,      # gate spacing of GPM
    "gr_file": gr2gpm_file,
}}

In [6]:
# Set parameters for this procedure
bw_sr = 0.71                  # SR beam width
platf = "gpm"                 # SR platform/product: one out of ["gpm", "trmm"]
zt = sr_pars[platf]["zt"]     # SR orbit height (meters)
dr_sr = sr_pars[platf]["dr"]  # SR gate length (meters)
gr_file = sr_pars[platf]["gr_file"]




In [7]:
def read_gr_boxpol(filename, loaddata=True):
    """
    Read ground radar boxpol scans
    
    in arbeit....
    
    """
    sdate = []
    refl = []
    
    gr_data = wrl.io.read_generic_netcdf(gr_file)
    grdata, grattrs = wrl.io.read_GAMIC_hdf5(gr_file)
    
    print gr_file
    
    dat_ = gr_data['what']['date']
    #print dat_
    #dat = dat_[0:10]
    #tim = dat_[11:-1]
    #print (dat, tim)
    # '2014-10-07T02:42:02Z'
    #date = dt.datetime.strptime(dat + tim, "%Y-%d-%m%H:%M:%S")
    #source = gr_data['what']['source']

    lon = gr_data['where']['lon']
    lat = gr_data['where']['lat']
    alt = gr_data['where']['height']

    import glob
    c_liste = glob.glob(gr_file.replace('n_ppi_010deg','*ppi*'))
    print c_liste
    ntilt = len(c_liste)
    
    ref_data = [[]]*ntilt
    rho_data = [[]]*ntilt


    print ('ntilit:',ntilt)    
    ngate = np.zeros(ntilt, dtype=np.int16)
    nbeam = np.zeros(ntilt)
    elang = np.zeros(ntilt)
    r0 = np.zeros(ntilt)
    dr = np.zeros(ntilt)
    a0 = np.zeros(ntilt)
    
    for jj in range(ntilt):
        print c_liste[jj]
        
        gr_data = wrl.io.read_generic_netcdf(c_liste[jj])
        grdata, grattrs = wrl.io.read_GAMIC_hdf5(c_liste[jj])

        #for i in range(0, ntilt):
        dset = gr_data['scan0']
        # Azimuth Start
        a0[jj] = dset['how']['azi_start'] 
        # Elevations Winkel
        elang[jj] = dset['how']['elevation']
        # Anzahl bins
        ngate[jj] = dset['how']['bin_count']
        # Start und Step der range
        r0[jj] = dset['how']['range_start']
        dr[jj] = dset['how']['range_step'] * dset['how']['range_samples']
        
        # Anzahl Rays
        nbeam[jj] = dset['how']['ray_count']

        #print ('azi start: ', a0)    
        #print ('elevation: ', elang)    
        #print ('bin count: ', ngate)    
        #print ('range start: ', r0)    
        #print ('range step: ', dr)    
        #print ('ray count: ', nbeam)    


        gr_dict = {}
        gr_dict.update({'date': dat_, 'lon': lon, 'lat': lat,
                        'alt': alt, 'ngate': ngate, 'nbeam': nbeam, 'ntilt': ntilt,
                        'r0': r0, 'dr': dr, 'a0': a0, 'elang': elang})
        if not loaddata:
            return gr_dict



        #for i in range(0, ntilt):
            #dset = gr_data['scan0']
            #data = dset['variables']['moment_10']['data']
        #dh = grdata['SCAN0']['ZH']['data']
        #dv = grdata['SCAN0']['ZV']['data']
        #data = (dh + dv)/2.
        
        data = grdata['SCAN0']['ZH']['data']
        
        
        rho = grdata['SCAN0']['RHOHV']['data']
        

        #refl.append(data)
        ref_data[jj]=data
        rho_data[jj]=rho
        #ref_data[ref_data<=0]=np.nan

        #refl1 = np.array(refl)
    gr_dict.update({'refl': ref_data})
    gr_dict.update({'rho': rho_data})
    
    return gr_dict, ntilt



In [8]:
# read matching GR data
gr_data, ntilt = read_gr_boxpol(gr_file)


/automount/radar-archiv/scans/2014/2014-10/2014-10-07/n_ppi_010deg/2014-10-07--02:37:44,00.mvol
['/automount/radar-archiv/scans/2014/2014-10/2014-10-07/n_ppi_010deg/2014-10-07--02:37:44,00.mvol', '/automount/radar-archiv/scans/2014/2014-10/2014-10-07/n_ppi_045deg/2014-10-07--02:37:44,00.mvol', '/automount/radar-archiv/scans/2014/2014-10/2014-10-07/n_ppi_082deg/2014-10-07--02:37:44,00.mvol', '/automount/radar-archiv/scans/2014/2014-10/2014-10-07/n_ppi_110deg/2014-10-07--02:37:44,00.mvol', '/automount/radar-archiv/scans/2014/2014-10/2014-10-07/n_ppi_140deg/2014-10-07--02:37:44,00.mvol', '/automount/radar-archiv/scans/2014/2014-10/2014-10-07/n_ppi_180deg/2014-10-07--02:37:44,00.mvol', '/automount/radar-archiv/scans/2014/2014-10/2014-10-07/n_ppi_280deg/2014-10-07--02:37:44,00.mvol', '/automount/radar-archiv/scans/2014/2014-10/2014-10-07/ppi_1p5deg/2014-10-07--02:37:44,00.mvol', '/automount/radar-archiv/scans/2014/2014-10/2014-10-07/ppi_2p4deg/2014-10-07--02:37:44,00.mvol', '/automount/rada

In [9]:
"""for i in range(len(gr_data.keys())):

    print (gr_data.keys()[i])
    print(gr_data[gr_data.keys()[i]])
    print ('------------------------- \n \n \n')
"""

"for i in range(len(gr_data.keys())):\n\n    print (gr_data.keys()[i])\n    print(gr_data[gr_data.keys()[i]])\n    print ('------------------------- \n \n \n')\n"

# Beginn der QnD shitty For schleife

In [10]:
SR, GR =  np.array([]), np.array([])
X, Y, Z = np.array([]), np.array([]), np.array([])

GRnum, GRstd =  np.array([]), np.array([])

gr_nr_elev, gr_ele = np.array([]), np.array([])


for ijk in range(ntilt):
#for ijk in range(1,6,2):

    ######### GR data and att
    ee = ijk
    # number of rays in gr sweep
    nray_gr = gr_data['nbeam'].astype("i4")[ee]
    # number of gates in gr beam
    ngate_gr = gr_data['ngate'].astype("i4")[ee]
    # number of sweeps
    nelev = gr_data['ntilt']
    # elevation of sweep (degree)
    elev_gr = gr_data['elang'][ee]
    # gate length (meters)
    dr_gr = gr_data['dr'][ee]
    # sweep datetime stamp
    #date_gr = gr_data['sdate'][ee]
    # range of first gate
    r0_gr = gr_data['r0'][ee]
    # azimuth angle of first beam
    a0_gr = gr_data['a0'][ee]
    # Longitude of GR
    lon0_gr = gr_data['lon']
    # Latitude of GR
    lat0_gr = gr_data['lat']
    # Altitude of GR (meters)
    alt0_gr = gr_data['alt']
    # Beam width of GR (degree)
    bw_gr = 1.###############################einlesen
    # reflectivity array of sweep
    ref_gr = gr_data['refl'][ee]
    # rhohv array of sweep
    rho_gr = gr_data['rho'][ee]
    
    if elev_gr<0.01:
        print ("Elevation zu niedrig")
        pass
    else:
        ############################################################################## Korrektur der GR Daten
        print ("----------------Elevationswinkel: ", elev_gr)
        print ("----------------number of rays in gr sweep: ", nray_gr)
        print ("----------------number of gates in gr beam: ", ngate_gr)
        print ("----------------number of sweeps: ", nelev)
        print ("----------------elevation of sweep: ", elev_gr)
        print ("----------------range of first gate: ", r0_gr)
        print ("----------------azimuth angle of first beam: ", a0_gr)
        print ("----------------Longitude of GR: ", lon0_gr)
        print ("----------------Latitude of GR: ", lat0_gr)
        print ("----------------Altitude of GR (meters): ",alt0_gr )
        print ("----------------gate length (meters): ",dr_gr )
        
        
        
        #print ("________Offset______")
        #Hardcoded....passt in etwa
        #ref_gr = ref_gr + 2


        #print ("________ATTCORR______")
        
        """        pia_harrison = wrl.atten.correctAttenuationHB(
            ref_gr,
            coefficients = dict(a=4.57e-5, b=0.731, gate_length=1.0),
            mode="warn",
            thrs=59.)
        pia_harrison[pia_harrison > 4.8] = 4.8

        #print ("________ATTCORR2______")
        ref_gr = ref_gr + pia_harrison"""


        #print ("________CLUTTER______")
        #rho_th  = 0.85
        rho_th = 0.9
        ref_gr[rho_gr<= rho_th] = np.nan
    
        print ("________TH______")
        #TH = 0
        #ref_gr[ref_gr<=TH]=np.nan
        
        
        
        coord = wrl.georef.sweep_centroids(nray_gr, dr_gr, ngate_gr, elev_gr)
        coords = wrl.georef.spherical_to_proj(coord[..., 0],
                                              np.degrees(coord[..., 1]),
                                              coord[..., 2],
                                              (lon0_gr, lat0_gr, alt0_gr))
        lon = coords[..., 0]
        lat = coords[..., 1]
        alt = coords[..., 2]
        bbox = wrl.zonalstats.get_bbox(lon, lat)
        print("Radar bounding box:\n\t%.2f\n%.2f           %.2f\n\t%.2f" %
              (bbox['top'], bbox['left'], bbox['right'], bbox['bottom']))
    

        # Calculate equivalent earth radius
        wgs84 = wrl.georef.get_default_projection()
        re1 = wrl.georef.get_earth_radius(lat0_gr, wgs84)
        #print("Earth radius 1:", re1)
        a = wgs84.GetSemiMajor()
        b = wgs84.GetSemiMinor()
        #print("SemiMajor, SemiMinor:", a, b)

        # Set up aeqd-projection gr-centered
        rad = wrl.georef.proj4_to_osr(('+proj=aeqd +lon_0={lon:f} ' +
                                           '+lat_0={lat:f} +a={a:f} ' +
                                           '+b={b:f}').format(lon=lon0_gr,
                                                              lat=lat0_gr,
                                                              a=a, b=b))
        re2 = wrl.georef.get_earth_radius(lat0_gr, rad)
        #print("Earth radius 2:", re2)

        # create gr range and azimuth arrays
        rmax_gr = r0_gr + ngate_gr * dr_gr
        r_gr = np.arange(0, ngate_gr) * dr_gr + dr_gr/2.
        az_gr = np.arange(0, nray_gr) - a0_gr
        #print("Range/Azi-Shape:", r_gr.shape, az_gr.shape)

        # create gr polar grid and calculate aeqd-xyz coordinates
        gr_polargrid = np.meshgrid(r_gr, az_gr)
        gr_xyz, rad = wrl.georef.spherical_to_xyz(gr_polargrid[0], gr_polargrid[1], elev_gr, (lon0_gr, lat0_gr, alt0_gr ))
        #print("XYZ-Grid-Shape:", gr_xyz.shape)

        # create gr poygon array in aeqd-xyz-coordinates
        gr_poly, rad1 = wrl.georef.spherical_to_polyvert(r_gr, az_gr, elev_gr, (lon0_gr, lat0_gr, alt0_gr))
        
        #print(gr_poly.shape, 360 * 600)
        gr_poly.shape = (nray_gr, ngate_gr, 5, 3)

        # get radar domain (outer ring)
        gr_domain = gr_xyz[:,-1,0:2]
        gr_domain = np.vstack((gr_domain, gr_domain[0]))


        # GR pulse volumes
        #   along one beam
        vol_gr = wrl.qual.pulse_volume(r_gr, dr_gr, bw_gr)
        #   with shape (nray_gr, ngate_gr)
        vol_gr = np.repeat(vol_gr, nray_gr).reshape((nray_gr, ngate_gr), order="F")


        
        X = np.append(X,gr_xyz[...,0])
        Y = np.append(Y,gr_xyz[...,1])
        Z = np.append(Z,gr_xyz[...,2])

        GR = np.append(GR, ref_gr)


        gr_nr_elev = np.append(gr_nr_elev, ref_gr.shape[0]*ref_gr.shape[1])
        gr_ele = np.append(gr_ele, elev_gr)



    

    
    

('----------------Elevationswinkel: ', 0.999755859375)
('----------------number of rays in gr sweep: ', 360)
('----------------number of gates in gr beam: ', 1000)
('----------------number of sweeps: ', 10)
('----------------elevation of sweep: ', 0.999755859375)
('----------------range of first gate: ', 0.0)
('----------------azimuth angle of first beam: ', 0.0)
('----------------Longitude of GR: ', 7.071663)
('----------------Latitude of GR: ', 50.73052)
('----------------Altitude of GR (meters): ', 99.5)
('----------------gate length (meters): ', 150.0)
________TH______
Radar bounding box:
	52.08
4.95           9.19
	49.38
('----------------Elevationswinkel: ', 4.4989013671875)
('----------------number of rays in gr sweep: ', 360)
('----------------number of gates in gr beam: ', 1000)
('----------------number of sweeps: ', 10)
('----------------elevation of sweep: ', 4.4989013671875)
('----------------range of first gate: ', 0.0)
('----------------azimuth angle of first beam: ', 0.0

In [11]:
pl.scatter(X[0:360000],Y[0:360000],c=GR[0:360000], edgecolors='none', vmin=0)
pl.colorbar()
pl.show()
print(gr_nr_elev)

[360000. 360000. 396000. 360000. 288000. 198000. 126000. 360000. 360000.
 720000.]


In [12]:
def get_my_cmap2():
    import matplotlib.cm as cm
    my_cmap = cm.get_cmap('jet',40)
    my_cmap.set_under('white')
    my_cmap.set_over('darkred')
    return my_cmap

def subplotcfad(Z,GR):
    import matplotlib.pyplot as plt
    m1 = ~np.isnan(Z) & ~np.isnan(GR)

    plt.hist2d(GR[m1], Z[m1],bins=120, cmap=get_my_cmap2(), vmin=0.1)#, vmax=2700)
    #plt.hexbin(GR[m1], Z[m1],bins=300, cmap=get_my_cmap2(), vmin=0.1)#, vmax=2700)
    plt.colorbar()
    plt.xlabel(r'$Z_H$')
    plt.ylabel('Height in m')

    plt.legend(loc='upper right', fontsize=10)
    #plt.grid(color='white')

    plt.ylim(0,7200)
    plt.xlim(0,50.)




In [ ]:
%%time
# Elevation unterteilen mit Cumulativer Summe
e_pos = gr_nr_elev.cumsum().astype(int)
from pcc import get_miub_cmap


fig = pl.figure(figsize=(18,6))
#plt.suptitle('CFAD 20141007')
for eee in range(len(gr_nr_elev)):
    ax1 = fig.add_subplot(2,5,eee+1)
    
    if eee==0: 
        subplotcfad(Z[0:e_pos[0]],GR[0:e_pos[0]])
        #pl.legend(loc='upper left')
        pl.title('Elevation: '+str(gr_ele[eee])[0:4])

    else:
        subplotcfad(Z[e_pos[eee-1]:e_pos[eee]],
                   GR[e_pos[eee-1]:e_pos[eee]])
        #pl.legend(loc='upper left')
        pl.title('Elevation: '+ str(gr_ele[eee])[0:4])


pl.tight_layout()
#pl.savefig('/automount/ftp/velibor/validation/boxpol_cfad_all_elevation_zdr.pdf')
#pl.savefig('/automount/ftp/velibor/validation/boxpol_cfad_all_elevation_zdr.png')

pl.show()

CPU times: user 3.67 s, sys: 2.1 s, total: 5.77 s
Wall time: 5 s
